# Greeks

Conceptually, there is nothing special about greeks for options on futures compared to other options.

As mentioned in the other options notebook, perhaps a point of confusion is that the delta and gamma
are with respect to the underlying futures price because the trader will presumably be using
the future to hedge their position (as opposed to physical crude oil, all the stocks in the S&P
500, soybeans, gold bars, etc.).

The cost-of-carry theory allows you to replace the futures price in Black's formula with
a spot price, but this is not useful in practice. In fact, it is often the case (see the
SVI references for example) that non-futures markets use the forward price to do their
analysis because it is a simplification, even though those other markets would not
compute deltas in terms of the forward.

This notebook focuses on some practical issues of managing
option risk rather than anything specific to options on futures.
The section on rho is the only one pointing out a key difference between options on futures
and options on equities.


## Skew adjustments

Professor Hendricks told me that you have already seen how to adjust greeks for skew,
so I will not belabor the point. After the skew notebook, however, it seems like a
good time to point out how bad things can happen in theory and practice.

Assuming your pricing function and skew function are differentiable,
the skew-adjusted delta is
$$\frac{dC}{dF} = \frac{\partial C}{\partial F} + \frac{\partial C}{\partial \sigma} \frac{\partial \sigma}{\partial F}.$$

How much that changes the naive $\frac{\partial C}{\partial F}$ depends on what assumptions you build into
how your skew curve will move with the futures price: $\frac{\partial \sigma}{\partial F}$.

Some simple examples include:
* Fixed skew/sticky strike: $\frac{\partial \sigma}{\partial F} = 0$
* Floating skew/sticky(ish) delta: $\frac{\partial \sigma}{\partial F} = - \sigma^\prime$
assuming the skew moves relative to an initial $F_0$ as $\sigma(K-(F-F_0))$
* Log-strike: $\frac{\partial \sigma}{\partial F} = - \sigma^\prime/F$ based on $\log(K/F)$ like SVI.
* Some models build this in with other features, so it would depend on the whole model you use.

With the exception of fixed skew, the dependence on $\sigma^\prime$ makes the skew-adjusted delta
sensitive to the smoothness of the skew model.

In [1]:
from dataclasses import dataclass

import databento as db
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from finm37000 import (
    OptionType,
    add_vol_plot,
    aggregate_ohlcv,
    calc_american_price,
    calc_black,
    calc_black_numerical_theta,
    calc_black_one_day_theta,
    calc_black_scholes_numerical_rho,
    calc_numerical_delta,
    calc_numerical_rho,
    calc_numerical_theta,
    calc_numerical_vega,
    calculate_option_vols,
    filter_otm,
    fit_raw_svi,
    fit_weighted_piecewise_polynomial_skew,
    get_databento_api_key,
    get_options_chain,
    get_top_of_book,
    temp_env,
    tz_chicago,
)

with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [2]:
interest_rate = 0.04
vol_target = "iv_midprice"
time_range = (
    (
        pd.Timestamp("2025-10-22T08:59:00", tz=tz_chicago),
        pd.Timestamp("2025-10-22T09:00:00", tz=tz_chicago),
    ),
    (
        pd.Timestamp("2025-10-23T12:59:00", tz=tz_chicago),
        pd.Timestamp("2025-10-23T13:00:00", tz=tz_chicago),
    ),
)
selected_time = time_range[0][1]
volume_start = pd.Timestamp("2025-10-22T07:00:00", tz=tz_chicago)
volume_end = pd.Timestamp("2025-10-22T16:00:00", tz=tz_chicago)

In [3]:
parent_option = "LO"
underlying_symbol = "CLF6"
option_label = "LOF6"
tick_size = 0.01
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=time_range[0][0],
    client=client,
)
top_prices = {}
for start, end in time_range:
    top_prices[end] = get_top_of_book(
        symbols=[underlying_symbol, *options_chain["raw_symbol"]],
        start=start,
        end=end,
        client=client,
    )

trades = client.timeseries.get_range(
    dataset=db.Dataset.GLBX_MDP3,
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=volume_start,
    end=volume_end,
    schema="trades",
).to_df()
trade_volume = aggregate_ohlcv(trades)

In [4]:
underlying_price = {}
otm_options = {}
all_options = {}
for timestamp, top_price in top_prices.items():
    with_vol, underlying_price[(underlying_symbol, timestamp)] = calculate_option_vols(
        top_price, underlying_symbol, options_chain, interest_rate
    )
    otm_options[(option_label, timestamp)] = filter_otm(
        with_vol, underlying_price[(underlying_symbol, timestamp)]
    )
    all_options[(option_label, timestamp)] = with_vol
df = otm_options[(option_label, selected_time)]
atm = underlying_price[(underlying_symbol, selected_time)]
df["log_strike"] = np.log(df["strike_price"] / atm)
df["total_variance"] = df["iv_midprice"] ** 2 * df["years_to_expiration"]
raw_svi = fit_raw_svi(df["log_strike"], df["total_variance"])
df["raw_svi"] = raw_svi.calc(df["log_strike"])
df["raw_svi_vol"] = (df["raw_svi"] / df["years_to_expiration"]) ** 0.5

piecewise_skew = fit_weighted_piecewise_polynomial_skew(
    df["strike_price"], df[vol_target], atm=atm, degree=4
)
df["piecewise_skew_deg4"] = piecewise_skew(df["strike_price"])

df["total_midprice_variance"] = df["iv_midprice"] ** 2 * df["years_to_expiration"]
df["piecewise_total_variance_deg4"] = (
    df["piecewise_skew_deg4"] ** 2 * df["years_to_expiration"]
)

In [5]:
def make_floating_skew(skew, original_underlying):
    """Transform function of strike to one of strike and relative underlying."""

    def floating(f, k):
        return skew(k - (f - original_underlying))

    return floating


def rescale_to_vol(svi_skew, t):
    def rescaled(log_k):
        return (svi_skew.calc(log_k) / t) ** 0.5

    return rescaled


def make_floating_svi(svi_skew, t):
    """Transform log-strike-to-total-variance siv to strike-vol."""
    vol_skew = rescale_to_vol(svi_skew, t)

    def floating(f, k):
        return vol_skew(log_k=np.log(k / f))

    return floating

In [6]:
floating_piecewise = make_floating_skew(piecewise_skew, original_underlying=atm)
slice_time = df["years_to_expiration"].unique()
assert len(slice_time) == 1
floating_svi = make_floating_svi(raw_svi, t=slice_time)

In [7]:
def approximate_slope(x, y):
    return np.diff(y, prepend=np.nan) / np.diff(x, prepend=np.nan)


df["piecewise_slope"] = approximate_slope(
    y=df["piecewise_skew_deg4"], x=df["strike_price"]
)
df["svi_slope"] = approximate_slope(y=df["raw_svi_vol"], x=df["strike_price"])

In [8]:
def calc_skew_delta(skew, future, strike, r, t, option_type, dfuture):  # noqa: PLR0913
    vol_up = skew(f=future + dfuture, k=strike)
    up = calc_american_price(
        future=future + dfuture,
        strike=strike,
        r=r,
        t=t,
        vol=vol_up,
        option_type=option_type,
    )
    vol_down = skew(f=future - dfuture, k=strike)
    down = calc_american_price(
        future=future - dfuture,
        strike=strike,
        r=r,
        t=t,
        vol=vol_down,
        option_type=option_type,
    )
    return (up - down) / (2.0 * dfuture)


df["svi_call_delta"] = calc_skew_delta(
    floating_svi,
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    option_type="C",
    dfuture=tick_size,
)
df["svi_put_delta"] = calc_skew_delta(
    floating_svi,
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    option_type="P",
    dfuture=tick_size,
)
df["piecewise_call_delta"] = calc_skew_delta(
    floating_piecewise,
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    option_type="C",
    dfuture=tick_size,
)
df["piecewise_put_delta"] = calc_skew_delta(
    floating_piecewise,
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    option_type="P",
    dfuture=tick_size,
)
df["call_delta"] = calc_numerical_delta(
    future=atm,
    strike=df["strike_price"],
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    r=interest_rate,
    option_type="C",
    dfuture=tick_size,
)
df["put_delta"] = calc_numerical_delta(
    future=atm,
    strike=df["strike_price"],
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    r=interest_rate,
    option_type="P",
    dfuture=tick_size,
)
df["call_vega_raw"] = calc_numerical_vega(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="C",
    dvol=0.0001,
)
df["put_vega_raw"] = calc_numerical_vega(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="P",
    dvol=0.0001,
)
df["put_vega"] = df["put_vega_raw"] * 0.01
df["call_vega"] = df["call_vega_raw"] * 0.01
dt = 1 / 365
df["call_theta_raw"] = calc_numerical_theta(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="C",
    dt=dt,
)
df["put_theta_raw"] = calc_numerical_theta(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="P",
    dt=dt,
)
df["put_theta"] = df["put_theta_raw"] * dt
df["call_theta"] = df["call_theta_raw"] * dt
df["call_rho_raw"] = calc_numerical_rho(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="C",
)
df["put_rho_raw"] = calc_numerical_rho(
    future=atm,
    strike=df["strike_price"],
    r=interest_rate,
    t=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type="P",
)
dr = 0.0001
df["put_rho"] = df["put_rho_raw"] * dr
df["call_rho"] = df["call_rho_raw"] * dr
df["call_bs_rho_raw"] = calc_black_scholes_numerical_rho(
    S=atm,
    K=df["strike_price"],
    r=interest_rate,
    q=0,
    T=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type=OptionType.CALL,
)
df["put_bs_rho_raw"] = calc_black_scholes_numerical_rho(
    S=atm,
    K=df["strike_price"],
    r=interest_rate,
    q=0,
    T=df["years_to_expiration"],
    vol=df["iv_midprice"],
    option_type=OptionType.PUT,
)
df["put_bs_rho"] = df["put_bs_rho_raw"] * dr
df["call_bs_rho"] = df["call_bs_rho_raw"] * dr

In [9]:
mid_range = (10, 180)
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"SVI Slope {option_label}",
    y_col="svi_slope",
    strike_range=mid_range,
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Piecewise Slope {option_label}",
    y_col="piecewise_slope",
    strike_range=mid_range,
)

fig.update_layout(
    title="Skew slope",
    xaxis_title="Strike Price",
    yaxis_title="Slope",
    template="plotly_white",
)
fig.show()

In [10]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"SVI Call Delta {option_label}",
    y_col="svi_call_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Piecewise Call Delta {option_label}",
    y_col="piecewise_call_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Unadjusted Call Delta {option_label}",
    y_col="call_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"SVI Put Delta {option_label}",
    y_col="svi_put_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Piecewise Put Delta {option_label}",
    y_col="piecewise_put_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Unadjusted Put Delta {option_label}",
    y_col="put_delta",
    strike_range=mid_range,
    mode="lines",
)

fig.update_layout(
    title="Skew Adjusted Delta",
    xaxis_title="Strike Price",
    yaxis_title="Delta",
    template="plotly_white",
)
fig.show()

In [11]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"SVI Call Delta {option_label}",
    y_col="svi_call_delta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name=f"Unadjusted Call Delta {option_label}",
    y_col="call_delta",
    strike_range=mid_range,
    mode="lines",
)


fig.update_layout(
    title="Skew Adjusted Delta",
    xaxis_title="Strike Price",
    yaxis_title="Delta",
    template="plotly_white",
)
fig.show()

## Units

An important feature to be aware of with options trading, whether using software or communicating with traders,
is that the mathematical or numerical values of a concept are often not in natural units for trading.

### POLL [Vega](https://www.polleverywhere.com/multiple_choice_polls/Qo8GeRRVqDVr1UvObCIke)

### Vega

The main feature to be aware of with vega is that $\frac{\partial C}{\partial \sigma}$ has units of price change per
unit change in volatility, but a unit change in volatility is $100\%$, which is not a useful scale for understanding
risk.

Instead, the units are often in terms of a $1\%$ change in vol, which is a simple rescaling.

In [12]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Call Vega",
    y_col="call_vega",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Put Vega",
    y_col="put_vega",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Vega by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Vega",
    template="plotly_white",
)
fig.show()

In [13]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Call Vega",
    y_col="call_vega_raw",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Put Vega",
    y_col="put_vega_raw",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Raw Vega by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Raw Vega",
    template="plotly_white",
)
fig.show()

Why don't the put vega and the call vega match? Can you intuitively understand which one is greater than the other?

### Theta

What makes theta different from other greeks?

As with vega, the natural units for calculation are not on a scale that makes sense for trading: price change per year.

Instead, traders understand theta as the 1-day time decay of their option position. Converting from years to days
depends on your day count convention, usually $1/252$, $1/360$, or $1/365$.

Moreover, there is no consensus about whether to present theta as negative or positive. Typically, option values decay
with time (when does it not?), so calculating the derivative gives a negative value, but in some trading environments
it is customary to measure it with the opposite sign.

In [14]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Call Theta",
    y_col="call_theta",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Put Theta",
    y_col="put_theta",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Theta by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Theta",
    template="plotly_white",
)
fig.show()

In [15]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Call Theta",
    y_col="call_theta_raw",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Put Theta",
    y_col="put_theta_raw",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Raw Theta by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Raw Theta",
    template="plotly_white",
)
fig.show()

### Theta near expiration

There is an important distinction between theta as a time derivative and theta as the one-day risk that
is especially noticeable near expiration.

N.B., I switch to the Black model here as the QuantLib American pricers are not great on small time divisions
needed for numerical theta calculations (or at least that's my best guess about why they sometimes
break). The same issue appears in American pricers.

In [16]:
strike = 60.0
vol = 0.32
one_strike = pd.DataFrame(
    {
        "strike_price": strike,
        "days_to_expiration": np.arange(50, 0, -0.25),
        "vol": vol,
    }
)
one_strike["years_to_expiration"] = one_strike["days_to_expiration"] / 365
one_strike["call_theo"] = calc_black(
    F=atm,
    K=one_strike["strike_price"],
    r=interest_rate,
    T=one_strike["years_to_expiration"],
    vol=one_strike["vol"],
    option_type=OptionType.CALL,
)
one_strike["theta"] = (
    calc_black_numerical_theta(
        F=atm,
        K=one_strike["strike_price"],
        r=interest_rate,
        T=one_strike["years_to_expiration"],
        vol=one_strike["vol"],
        option_type=OptionType.CALL,
        dt=0.0001,
    )
    * 1
    / 365
)
one_strike["one_day_theta"] = calc_black_one_day_theta(
    F=atm,
    K=one_strike["strike_price"],
    r=interest_rate,
    T=one_strike["years_to_expiration"],
    vol=one_strike["vol"],
    option_type=OptionType.CALL,
    dt=1 / 365,
)
one_day_tangent = pd.DataFrame(
    {
        "days_to_expiration": one_strike["days_to_expiration"],
        "years_to_expiration": one_strike["years_to_expiration"],
    }
)
tangent_day = 1
tangent_day_mask = np.isclose(one_strike["days_to_expiration"], tangent_day, 0.0001)
tangent_slope = one_strike[tangent_day_mask]["theta"].iloc[0]
tangent_y = one_strike[tangent_day_mask]["call_theo"].iloc[0]
one_day_tangent["tangent"] = (
    tangent_slope * (one_strike["days_to_expiration"] - tangent_day) + tangent_y
)

/Users/ericpatterson/.venv/finm37000-2025-09-17/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

invalid value encountered in sqrt



In [17]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=one_strike["days_to_expiration"],
        y=one_strike["theta"],
        mode="lines",
        name="Theta",
    )
)
fig.add_trace(
    go.Scatter(
        x=one_strike["days_to_expiration"],
        y=one_strike["one_day_theta"],
        mode="lines",
        name="One-day Theta",
    )
)
fig.update_layout(
    title=f"Theta over time: Strike {strike}, Vol {vol:.1%}",
    xaxis_title="Days to expiration",
    yaxis_title="Theta",
    template="plotly_white",
)
fig.show()

In [18]:
days_near_exp = 3
near_exp = one_strike[one_strike["days_to_expiration"] < days_near_exp]
near_exp_tangent = one_day_tangent[
    one_day_tangent["days_to_expiration"] < days_near_exp
]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=near_exp["days_to_expiration"],
        y=near_exp["theta"],
        mode="lines",
        name="Theta",
    )
)
fig.add_trace(
    go.Scatter(
        x=near_exp["days_to_expiration"],
        y=near_exp["one_day_theta"],
        mode="lines",
        name="One-day Theta",
    )
)
fig.add_trace(
    go.Scatter(
        x=near_exp["days_to_expiration"],
        y=near_exp["call_theo"],
        mode="lines",
        name="Call price",
    )
)
fig.add_trace(
    go.Scatter(
        x=near_exp_tangent["days_to_expiration"],
        y=near_exp_tangent["tangent"],
        mode="lines",
        name="T=1 day tangent",
        line=dict(dash="dash"),
    )
)
fig.update_layout(
    title=f"Theta over time: Strike {strike}, Vol {vol:.1%}",
    xaxis_title="Days to expiration",
    yaxis_title="Theta",
    template="plotly_white",
)
fig.show()

### Rho

Rho is a minor greek in the context of options on futures. In terms of the Black model for European options on futures,
the interest rate only appears in the discount factor. The underlying futures price is not growing at the risk-free
rate, so the interest rate dependence is minimal and less important compared to its role in the Black-Scholes model.

As with theta and vega, it is typical to rescale rho to a basis point change in the interest rate rather than a $100$
basis point change in the typical scale for $r$.

In [19]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Call Rho",
    y_col="call_rho",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Put Rho",
    y_col="put_rho",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Rho by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Rho",
    template="plotly_white",
)
fig.show()

In [20]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Call Rho",
    y_col="call_rho_raw",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Raw Put Rho",
    y_col="put_rho_raw",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Raw Rho by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Raw Rho",
    template="plotly_white",
)
fig.show()

In [21]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="American Futures Call Rho",
    y_col="call_rho",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Black-Scholes Call Rho",
    y_col="call_bs_rho",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Rho by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Rho",
    template="plotly_white",
)
fig.show()

In [22]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="American Futures Put Rho",
    y_col="put_rho",
    strike_range=mid_range,
    mode="lines",
)
add_vol_plot(
    fig=fig,
    vol_df=df,
    name="Black-Scholes Put Rho",
    y_col="put_bs_rho",
    strike_range=mid_range,
    mode="lines",
)
fig.update_layout(
    title="Rho by Strike",
    xaxis_title="Strike Price",
    yaxis_title="Rho",
    template="plotly_white",
)
fig.show()

## Option spreads

Many option trades are done with spreads, and it is important to move beyond looking at the risk
profiles of calls or puts and look at the risk profile for the spread to understand how
trades will affect your position.

Useful terminology reference:
https://www.ice.com/publicdocs/technology/ICE_Strategy_Code_Reference_Manual.pdf


N.B., I am not going to pursue extracting option spread data from Databento for CME products.
The process for resolving the spreads to symbols is nontrivial:
https://cmegroupclientsite.atlassian.net/wiki/spaces/EPICSANDBOX/pages/457096757/UDS+-+Process+to+Build+a+Display+Name

Recall CME spread symbol conventions:
https://cmegroupclientsite.atlassian.net/wiki/spaces/EPICSANDBOX/pages/457089763/Spreads+and+Combinations+Available+on+CME+Globex

We can see the symbols for the spreads in the definition table, but you need to build a way to find what you are looking
for.

In [23]:
with_spreads = get_options_chain(
    parent=parent_option,
    underlying=None,
    instrument_class=None,
    start=time_range[0][0],
    client=client,
)

In [24]:
spread_cols = [
    "symbol",
    "secsubtype",
]
with_spreads[with_spreads["instrument_class"] == "T"][spread_cols]

,symbol,secsubtype
ts_recv,,
2025-10-22 00:00:00+00:00,UD:1N: VT 2555838,VT
2025-10-22 00:00:00+00:00,UD:1N: VT 2555867,VT
2025-10-22 00:00:00+00:00,UD:1N: SG 2541900,SG
2025-10-22 00:00:00+00:00,UD:1N: VT 2528755,VT
2025-10-22 00:00:00+00:00,UD:1N: GN 2528727,GN
...,...,...
2025-10-22 23:41:01.373306260+00:00,UD:1N: GN 2596862,GN
2025-10-22 23:42:49.569581588+00:00,UD:1N: SG 2596880,SG
2025-10-22 23:43:09.443254224+00:00,UD:1N: SG 2596882,SG


### Straddle

Straddles are long a call and a put at the same strike.
The at-the-money straddle is the most liquid straddle reflecting the liquidity of the legs.

Many skew fitting and risk assessment frameworks calibrate overall volatility levels to the at-the-money straddle,
adding yet another meaning to the overused initialism ATM.
In other words, the ATM may sometimes implicitly refer to the ATM straddle (e.g., "I am short 50 ATMS"),
but it can also be used as a unit of measure for vega where vega at other strikes are recast relative
to the vega of the ATM straddle.

In other words, this spread is very common and very important, but the terminology can be
a bit fluid and you will need to adapt to the context you are in.

Looking at the vega profile of the straddle shows why it is used to adjust overall volatility.

In [25]:
@dataclass
class Put:
    strike: float

    def exercise_value(self, futures_price):
        """Calculate put exercise value."""
        return np.fmax(self.strike - futures_price, 0)


@dataclass
class Call:
    strike: float

    def exercise_value(self, futures_price):
        """Calculate call exercise value."""
        return np.fmax(futures_price - self.strike, 0)


def make_spread_payoff(legs, coefficients, futures_prices):
    payoff = np.zeros_like(futures_prices)
    for leg, coef in zip(legs, coefficients):
        payoff += coef * leg.exercise_value(futures_prices)
    return payoff

In [26]:
def make_prospective_spread(  # noqa: PLR0913
    instrument_df, leg_symbols, coefficients, skew, futures_prices, dfuture=0.01
):
    spread_legs = instrument_df[instrument_df["symbol"].isin(leg_symbols)]
    spread_legs = spread_legs.merge(futures_prices, how="cross")
    spread_legs["skew"] = skew(
        k=spread_legs["strike_price"], f=spread_legs["new_underlying_price"]
    )
    spread_legs["theo"] = calc_american_price(
        future=spread_legs["new_underlying_price"],
        strike=spread_legs["strike_price"],
        t=spread_legs["years_to_expiration"],
        r=interest_rate,
        vol=spread_legs["skew"],
        option_type=spread_legs["instrument_class"],
    )
    spread_legs["delta"] = calc_numerical_delta(
        future=spread_legs["new_underlying_price"],
        strike=spread_legs["strike_price"],
        r=interest_rate,
        t=spread_legs["years_to_expiration"],
        vol=spread_legs["skew"],
        option_type=spread_legs["instrument_class"],
        dfuture=dfuture,
    )
    spread_legs["skew_adjusted_delta"] = calc_skew_delta(
        skew,
        future=spread_legs["new_underlying_price"],
        strike=spread_legs["strike_price"],
        r=interest_rate,
        t=spread_legs["years_to_expiration"],
        option_type=spread_legs["instrument_class"],
        dfuture=dfuture,
    )
    spread_legs["true_vega"] = calc_numerical_vega(
        future=spread_legs["new_underlying_price"],
        strike=spread_legs["strike_price"],
        r=interest_rate,
        t=spread_legs["years_to_expiration"],
        vol=spread_legs["skew"],
        option_type=spread_legs["instrument_class"],
        dvol=0.0001,
    )
    spread_legs["vega"] = spread_legs["true_vega"] * 0.01
    spread = spread_legs.pivot_table(
        columns="symbol",
        index="new_underlying_price",
        values=["theo", "delta", "skew_adjusted_delta", "vega"],
    )
    spread["spread_theo"] = 0
    spread["spread_delta"] = 0
    spread["spread_skew_adjusted_delta"] = 0
    spread["spread_vega"] = 0
    for leg, coef in zip(leg_symbols, coefficients):
        spread["spread_theo"] += coef * spread[("theo", leg)]
        spread["spread_delta"] += coef * spread[("delta", leg)]
        spread["spread_skew_adjusted_delta"] += (
            coef * spread[("skew_adjusted_delta", leg)]
        )
        spread["spread_vega"] += coef * spread[("vega", leg)]
    return spread

In [27]:
straddle_strike = 58.5
straddle_symbols = ["LOF6 C5850", "LOF6 P5850"]
straddle_coefficients = [1, 1]
prospective_futures = pd.Series(np.arange(50, 70, 0.25), name="new_underlying_price")
straddle_legs = (
    Call(strike=58.5),
    Put(strike=58.5),
)
straddle_payoff = make_spread_payoff(
    legs=straddle_legs,
    coefficients=straddle_coefficients,
    futures_prices=prospective_futures,
)
straddle = make_prospective_spread(
    instrument_df=all_options[(option_label, selected_time)],
    leg_symbols=straddle_symbols,
    coefficients=straddle_coefficients,
    futures_prices=prospective_futures,
    skew=floating_svi,
)

In [28]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=prospective_futures,
        y=straddle_payoff,
        mode="lines",
    )
)
fig.add_vline(
    x=straddle_strike,
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Straddle strike={straddle_strike}",
    annotation_position="top right",
)
fig.update_layout(
    title="Straddle Terminal Payoff",
    xaxis_title="Underlying Price",
    yaxis_title="Spread Value",
    template="plotly_white",
)
fig.show()

In [29]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=straddle.index,
        y=straddle["spread_vega"],
        mode="lines",
        name="Straddle vega",
    )
)
fig.add_vline(
    x=straddle_strike,
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Straddle strike={straddle_strike}",
    annotation_position="bottom right",
)
fig.update_layout(
    title="Straddle vega",
    xaxis_title="Underlying Price",
    yaxis_title="Vega",
    template="plotly_white",
)
fig.show()

It is always important to understand the delta of a trade, but you can see that the put and call deltas significantly
cancel as long as the underlying is near the money making it fairly delta neutral ATM.
As the underlying moves, the delta of the ITM side dominates the position.

In [30]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=straddle.index,
        y=straddle["spread_delta"],
        mode="lines",
        name="Static delta",
    )
)
fig.add_trace(
    go.Scatter(
        x=straddle.index,
        y=straddle["spread_skew_adjusted_delta"],
        mode="lines",
        name="Skew-adjusted delta",
    )
)
fig.add_vline(
    x=straddle_strike,
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Straddle strike={straddle_strike}",
    annotation_position="top right",
)
fig.update_layout(
    title="Straddle delta",
    xaxis_title="Underlying Price",
    yaxis_title="Delta",
    template="plotly_white",
)
fig.show()

### Call spread

Call and put spreads are some of the most common spreads traded.
By taking two semi-offsetting positions, there will be some greek cancellation,
leading to more targeted adjustments to your risk.

In [31]:
call_spread_symbols = ["LOF6 C6000", "LOF6 C6500"]
call_spread_strikes = [60, 65]
call_spread_coefficients = [1, -1]
call_spread_legs = (
    Call(strike=60),
    Call(strike=65),
)
call_spread_payoff = make_spread_payoff(
    legs=call_spread_legs,
    coefficients=call_spread_coefficients,
    futures_prices=prospective_futures,
)
call_spread = make_prospective_spread(
    instrument_df=all_options[(option_label, selected_time)],
    leg_symbols=call_spread_symbols,
    coefficients=call_spread_coefficients,
    futures_prices=prospective_futures,
    skew=floating_svi,
)

In [32]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=prospective_futures,
        y=call_spread_payoff,
        mode="lines",
    )
)
fig.add_vline(
    x=call_spread_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low strike={call_spread_strikes[0]}",
    annotation_position="top left",
)
fig.add_vline(
    x=call_spread_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High strike={call_spread_strikes[1]}",
    annotation_position="top right",
)
fig.update_layout(
    title="Call Spread Terminal Payoff",
    xaxis_title="Underlying Price",
    yaxis_title="Spread Value",
    template="plotly_white",
)
fig.show()

In [33]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=call_spread.index,
        y=call_spread["spread_delta"],
        mode="lines",
        name="Static delta",
    )
)
fig.add_trace(
    go.Scatter(
        x=call_spread.index,
        y=call_spread["spread_skew_adjusted_delta"],
        mode="lines",
        name="Skew-adjusted delta",
    )
)
fig.add_vline(
    x=call_spread_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low call strike={call_spread_strikes[0]}",
    annotation_position="bottom left",
)
fig.add_vline(
    x=call_spread_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High call strike={call_spread_strikes[1]}",
    annotation_position="bottom right",
)
fig.update_layout(
    title="Call spread delta",
    xaxis_title="Underlying Price",
    yaxis_title="Delta",
    template="plotly_white",
)
fig.show()

In [34]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=call_spread.index,
        y=call_spread["spread_vega"],
        mode="lines",
        name="Call spread vega",
    )
)
fig.add_vline(
    x=call_spread_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low call strike={call_spread_strikes[0]}",
    annotation_position="bottom left",
)
fig.add_vline(
    x=call_spread_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High call strike={call_spread_strikes[1]}",
    annotation_position="top right",
)
fig.update_layout(
    title="Call spread vega",
    xaxis_title="Underlying Price",
    yaxis_title="Vega",
    template="plotly_white",
)
fig.show()

### Butterfly

Call and put butterflies use offsetting positions to measure risk in the wings.
They are often used to calibrate parameters that control underlying model or skew
kurtosis.

In [35]:
butterfly_symbols = ["LOF6 C6000", "LOF6 C6250", "LOF6 C6500"]
butterfly_strikes = [60, 62.5, 65]
prospective_futures = pd.Series(np.arange(50, 80, 0.25), name="new_underlying_price")
butterfly_coefficients = [1, -2, 1]
butterfly_legs = (
    Call(strike=60),
    Call(strike=62.5),
    Call(strike=65),
)
butterfly_payoff = make_spread_payoff(
    legs=butterfly_legs,
    coefficients=butterfly_coefficients,
    futures_prices=prospective_futures,
)
butterfly = make_prospective_spread(
    instrument_df=all_options[(option_label, selected_time)],
    leg_symbols=butterfly_symbols,
    coefficients=butterfly_coefficients,
    futures_prices=prospective_futures,
    skew=floating_svi,
)

In [36]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=prospective_futures,
        y=butterfly_payoff,
        mode="lines",
    )
)
fig.add_vline(
    x=butterfly_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low strike={butterfly_strikes[0]}",
    annotation_position="top left",
)
fig.add_vline(
    x=butterfly_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Mid strike={butterfly_strikes[1]}",
    annotation_position="bottom",
)
fig.add_vline(
    x=butterfly_strikes[2],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High strike={butterfly_strikes[2]}",
    annotation_position="top right",
)
fig.update_layout(
    title="Butterfly Terminal Payoff",
    xaxis_title="Underlying Price",
    yaxis_title="Spread Value",
    template="plotly_white",
)
fig.show()

In [37]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=butterfly.index,
        y=butterfly["spread_vega"],
        mode="lines",
        name="Call spread vega",
    )
)
fig.add_vline(
    x=butterfly_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low strike={butterfly_strikes[0]}",
    annotation_position="bottom left",
)
fig.add_vline(
    x=butterfly_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Mid strike={butterfly_strikes[1]}",
    annotation_position="bottom right",
)
fig.add_vline(
    x=butterfly_strikes[2],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High strike={butterfly_strikes[2]}",
    annotation_position="bottom right",
)
fig.update_layout(
    title="Butterfly vega",
    xaxis_title="Underlying Price",
    yaxis_title="Vega",
    template="plotly_white",
)
fig.show()

In [38]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=butterfly.index,
        y=butterfly["spread_delta"],
        mode="lines",
        name="Static delta",
    )
)
fig.add_trace(
    go.Scatter(
        x=butterfly.index,
        y=butterfly["spread_skew_adjusted_delta"],
        mode="lines",
        name="Skew-adjusted delta",
    )
)
fig.add_vline(
    x=butterfly_strikes[0],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Low strike={butterfly_strikes[0]}",
    annotation_position="bottom left",
)
fig.add_vline(
    x=butterfly_strikes[1],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"Mid strike={butterfly_strikes[1]}",
    annotation_position="top right",
)
fig.add_vline(
    x=butterfly_strikes[2],
    line_dash="dot",
    line_color="gray",
    annotation_text=f"High strike={butterfly_strikes[2]}",
    annotation_position="bottom right",
)
fig.update_layout(
    title="Butterfly delta",
    xaxis_title="Underlying Price",
    yaxis_title="Delta",
    template="plotly_white",
)
fig.show()